<a href="https://colab.research.google.com/github/SelmaDM/Pyterrier/blob/master/Re-Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTerrier - Neural Re-Ranking 

Dans ce TP vous allez :

 - reclasser des documents en utilisant des modèles neuronaux comme KNRM, Vanilla BERT, EPIC et monoT5.


# Installation


In [5]:
!pip install -q --upgrade python-terrier

### Installation des plugins Pyterrier  

Nous installons les plugins PyTerrier [OpenNIR](https://opennir.net/) et [monoT5](https://github.com/terrierteam/pyterrier_t5). Vous pouvez ignorer sans risque les erreurs de version des paquets.

In [6]:
!pip install -q --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install -q --upgrade git+https://github.com/terrierteam/pyterrier_t5

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


## Preliminary steps

Ces lignes sont nécessaires pour travailler avec DeepCT.

In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf


**[PyTerrier](https://github.com/terrier-org/pyterrier) initialization** 

In [8]:
import pyterrier as pt
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')


from pyterrier.measures import * # allow for natural measure names
import onir_pt

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


### Indexation

In [9]:
!rm -rf ./terrier_cord19/*

In [10]:
import os

pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv
docs_iter:   0%|                                    | 0/192509 s<?, ?doc/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 0.00/269M s<?, ?B/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.0%| 65.5k/269M s<10:34, 425kB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.1%| 270k/269M s<05:07, 877kB/s] 
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv: 0.4%| 1.06M/269M s<01:60, 2.24MB/s]
https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

<ipython-input-10-59cedea60614>:11: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  index_ref = indexer.index(cord19.get_corpus_iter(),


14:04:48.960 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
14:07:03.640 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.indexDocuments(BasicIndexer.java:270)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:388)
	at org.terrier.structures.indexing.Indexer.index(Inde

## Re-Rankers 

Commençons à explorer quelques méthodes neuronales de re-classement ! Nous pouvons les construire à partir de zéro en utilisant `onir_pt.reranker`.

Le modèle de re-ranking d'OpenNIR est composé de :
 - `ranker` (par exemple, `drmm`, `knrm`, ou `pacrr`). Ceci définit l'architecture neuronale de classement.
 - `vocab` (par exemple, `wordvec_hash`, ou `bert`). Ceci définit comment le texte est encodé par le modèle. Cette approche rend facile l'échange de différentes représentations de texte.

L'exécution de cette ligne prendra quelques minutes car elle télécharge et prépare les vecteurs de mots.

In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2023-03-17 14:07:30,030][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2023-03-17 14:07:46,294][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2023-03-17 14:07:46,306][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [15.68s] [682M] [36.2MB/s]
[2023-03-17 14:07:46,315][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [16.29s]
[2023-03-17 14:07:46,315][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2023-03-17 14:08:09,887][WordvecHashVocab][DEBUG] [finished] extracting vecs [23.57s]
[2023-03-17 14:08:09,888][WordvecHashVocab][DEBUG] [starting] loading vecs into memory


Voyons comment ces modèles fonctionnent pour le classement !

In [ ]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF") % 50
get_text = pt.text.get_text(cord19, 'abstract') #>> pt.apply.title_abstract(lambda r: r['title'] + ' ' + r['abstract'])


In [ ]:
topics = cord19.get_topics(variant='description')
qrels = cord19.get_qrels()

In [ ]:
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=SEED)


test_qrels = qrels # seulement les annotations des topics en réponse sont utilisés, donc pas de problème si on utilise tout
train_qrels = qrels
valid_qrels = qrels

In [ ]:

# build a sub-pipeline to get the concatenated title and abstract text
pipeline = tfidf >> get_text >> knrm
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

Cela ne fonctionne pas très bien car le modèle n'est pas entraîné ; il utilise des poids aléatoires pour combiner les scores de la matrice de similarité.

## Entraînement du re-ranker

Vous pouvez entraîner des modèles de re-classement dans PyTerrier en utilisant la méthode `fit`. 

In [ ]:
pipeline.fit(
    train_topics,
    train_qrels,
    valid_topics,
    valid_qrels)

In [ ]:
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> KNRM (trained)'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

Le résultat est plus élévé, mais cela reste moins bon que le TFIDF. Proposez une hypothèse sur le problème.

## Vanilla BERT

Les modèles linguistiques contextualisés, tels que [BERT] (https://arxiv.org/abs/1810.04805), sont des modèles neuronaux beaucoup plus puissants qui se sont avérés efficaces pour le classement.

Nous allons essayer d'utiliser une version "vanille" (ou "mono") du modèle BERT. Le modèle BERT est pré-entraîné pour la modélisation du langage et la prédiction de la phrase suivante.

In [ ]:
#del knrm # clear out memory from KNRM
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='abstract', vocab_config={'train': True})

Voyons comment ce modèle se comporte sur TREC COVID.

In [ ]:
pipeline = tfidf % 50 >> get_text >> vbert
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> VBERT'],
    baseline=0,
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

Comme nous le constatons, bien que le modèle soit pré-entraîné, il n'obtient pas de très bons résultats. Cela est dû au fait qu'il n'est pas réglé pour la tâche de classement par pertinence.

Cependant, nous pouvons entraîner le modèle pour le classement (comme indiqué ci-dessus pour KNRM).

## monoT5

Le modèle [monoT5](https://arxiv.org/abs/2003.06713) évalue les documents à l'aide d'un modèle de langage causal. Voyons comment cette approche fonctionne sur TREC COVID.

La classe `MonoT5ReRanker` de `pyterrier_t5` charge automatiquement une version du classeur monoT5 qui est entraînée sur le jeu de données MS MARCO passage.

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

In [ ]:
pipeline = (tfidf >> get_text >> monoT5)
pt.Experiment(
    [tfidf, pipeline],
    test_topics,
    qrels,
    names=['TFIDF', 'TFIDF >> T5'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10, "mrt"]
)

Comme on pouvait s'y attendre, les résultats sont bien meilleurs en termes de NDCG@10 (0.5958 vs 0.6855).

# Tâche pratique

Comme pour le TP precedent, utilisez les modèles implémentés pour cord19 dans une tâche de question-réponse. Dans ce contexte, les requêtes sont de questions et les documents sont des documents qui pourraient contenir la réponse. Notez que vous devez refaire l'indexation ainsi que les autres étapes étudiées dans ce TP. Vous pouvez le télécharger le dataset en utilisant les lines de code ci-dessous.

In [ ]:
fiqa = {}
fiqa['train'] = pt.datasets.get_dataset('irds:beir/fiqa/train')
fiqa['valid'] = pt.datasets.get_dataset('irds:beir/fiqa/dev')
fiqa['test'] = pt.datasets.get_dataset('irds:beir/fiqa/test')

test_topics = fiqa['test'].get_topics(variant='text')
test_qrels = fiqa['test'].get_qrels()

train_topics = fiqa['train'].get_topics(variant='text')
train_qrels = fiqa['train'].get_qrels()

valid_topics = fiqa['valid'].get_topics(variant='text')
valid_qrels = fiqa['valid'].get_qrels()

In [ ]:
# Creation des fichiers d'indexation
!rm -rf ./pd_index_fiqa_train
!rm -rf ./pd_index_fiqa_test
!rm -rf ./pd_index_fiqa_valid

In [ ]:
def indexingFun(data_list):
  pt_index_path_dict = {}
  for data in data_list:
    pt_index_path = './pd_index_fiqa' + '_' + data
    pt_index_path_dict[data] = pt_index_path

    if not os.path.exists(pt_index_path + "/data.properties"):
        # create the index, using the IterDictIndexer indexer 
        indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

        # we give the dataset get_corpus_iter() directly to the indexer
        # while specifying the fields to index and the metadata to record
        index_ref = indexer.index(fiqa[data].get_corpus_iter())

    else:
        # if you already have the index, use it.
        index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
  return pt_index_path_dict

In [ ]:
def tpPipeline(index_ref):
  tf = pt.BatchRetrieve(index_ref, wmodel="Tf")
  tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF")
  bm_25 = pt.BatchRetrieve(index_ref, wmodel='BM25')
  
  pipeline = ((tf %10) | (tfidf % 10)) >> bm_25

  return pipeline

In [ ]:
def tpCharactPipeline(pipeline, data, index_ref):
  return (pipeline) >> pt.text.get_text(data, metadata=['doc_id', 'text'], by_query=True) >> (
      pt.transformer.IdentityTransformer()
      ** 
      pt.BatchRetrieve(index_ref, wmodel="CoordinateMatch")
      )

In [ ]:
data_list = ['train', 'test', 'valid']

names = ['Pipeline', 'FastR', 'RFR', 'Lambda']
feature_names = ['CoordinateMatch']

# Indexation
pt_index_path_dict = indexingFun(data_list)

# Pipeline Creation
pipeline = tpPipeline(pt.IndexRef.of(pt_index_path_dict['train'] + "/data.properties"))
charact_pipeline = tpCharactPipeline(pipeline, fiqa['train'], pt_index_path_dict['train'])

In [ ]:
monoT5 = MonoT5ReRanker(text_field='text')

In [ ]:
pipeline_T5 = (charact_pipeline >> monoT5)

pt.Experiment(
    [pipeline, pipeline_T5],
    test_topics,
    test_qrels,
    names=['Pipeline', 'Pipeline >> T5'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10, "mrt"]
)

In [ ]:
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='text', vocab_config={'train': True})

In [ ]:
pipeline_VBERT = (charact_pipeline >> vbert)
pt.Experiment(
    [pipeline, pipeline_VBERT],
    test_topics,
    test_qrels,
    names=['Pipeline', 'Pipeline >> VBERT'],
    baseline=0,
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='text')

In [ ]:
pipeline_knrm = (charact_pipeline >> knrm)
pt.Experiment(
    [pipeline, pipeline_knrm],
    test_topics,
    test_qrels,
    names=['Pipeline', 'Pipeline >> KNRM'],
    eval_metrics=[AP(rel=2), nDCG, nDCG@10, P(rel=2)@10]
)

Identifiez-vous des avantages par rapport à l'utilisation de Learning to Rank ?